**Parent kernal [link ](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline-lb)**

**changes n_estimators=1000,num_leaves=500,max_depth=10**

In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto


TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

In [2]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [3]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


# Training

## Utility functions to train a model for one asset

## Loop over all assets

In [4]:
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras import *
from tensorflow.keras.optimizers import *
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['upper_Shadow'] = upper_shadow(df_feat)
    df_feat['lower_Shadow'] = lower_shadow(df_feat)
    df_feat["high_div_low"] = df_feat["High"] / df_feat["Low"]
    #df_feat["open_sub_close"] = df_feat["Open"] - df_feat["Close"]
    df_feat['trade']=df_feat['Close']-df_feat['Open']
    df_feat['gtrade']=df_feat['trade']/df_feat['Count']
    df_feat['shadow1']=df_feat['trade']/df_feat['Volume']
    #df_feat['shadow2']=df_feat['upper_Shadow']/df['Low']
    df_feat['shadow3']=df_feat['upper_Shadow']/df_feat['Volume']
    #df_feat['shadow4']=df_feat['lower_Shadow']/df['High']
    df_feat['shadow5']=df_feat['lower_Shadow']/df_feat['Volume']
    
    df_feat['diff1']=df_feat['Volume']-df_feat['Count']
    
    df_feat['mean1']=(df_feat['shadow5']+df_feat['shadow3'])/2
    
    df_feat['mean2']=(df_feat['shadow1']+df_feat['Volume'])/2
    
    df_feat['mean3']=(df_feat['trade']+df_feat['gtrade'])/2
    
    df_feat['mean4']=(df_feat['diff1']+df_feat['upper_Shadow'])/2
    
    df_feat['mean5']=(df_feat['diff1']+df_feat['lower_Shadow'])/2 
    
    return df_feat
def log(model,X_train, X_valid, y_train, y_valid,train_split=1.0):
    if train_split > 0:
        X_train=X_train[:int(train_split*X_train.shape[0])]
        y_train=y_train[:int(train_split*y_train.shape[0])]
    
        pred=model.predict(X_train)
        print('Training :- ')
        print(f'MSE : {np.mean((y_train-pred)**2)}')
        print(f'CV : {pearsonr(pred,y_train)[0]}')
    pred=model.predict(X_valid)
    print('Validation :- ')
    print(f'MSE : {np.mean((y_valid-pred)**2)}')
    print(f'CV : {pearsonr(pred,y_valid)[0]}')

def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
   
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    
    input=Input(shape=X.shape[1:])
    x=Dense(64,activation='relu')(input)
    x=Dense(64*2,activation='relu')(x)
    x=Dense(64*3,activation='relu')(x)
    x=Dense(64,activation='relu',name='end')(x)
    x=Dense(64*3,activation='relu')(x)
    x=Dense(64*2,activation='relu')(x)
    x=Dense(X.shape[1])(x)
    model=Model(input,x)
    model.compile(loss='mae',optimizer=Adam(0.0001),metrics=['mae'])
    model.fit(X,X,epochs=12,steps_per_epoch=10000,validation_split=0.2)
    Model(model.input,model.get_layer('end').output).save(f'model {asset_id}.h5')
    

In [5]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    get_Xy_and_model_for_asset(df_train, asset_id)    
    

Training model for Binance Coin     (ID=0 )


2021-11-12 06:47:55.240861: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 06:47:55.241900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 06:47:55.242651: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 06:47:55.243686: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/12
10000/10000 [==============================] - 46s 4ms/step - loss: 4.3592 - mae: 4.3592 - val_loss: 3.5414 - val_mae: 3.5414
Epoch 2/12
10000/10000 [==============================] - 49s 5ms/step - loss: 1.1442 - mae: 1.1442 - val_loss: 2.4271 - val_mae: 2.4271
Epoch 3/12
10000/10000 [==============================] - 44s 4ms/step - loss: 1.0716 - mae: 1.0716 - val_loss: 2.7953 - val_mae: 2.7953
Epoch 4/12
10000/10000 [==============================] - 48s 5ms/step - loss: 1.0418 - mae: 1.0418 - val_loss: 2.1458 - val_mae: 2.1458
Epoch 5/12
10000/10000 [==============================] - 44s 4ms/step - loss: 0.9684 - mae: 0.9684 - val_loss: 2.6106 - val_mae: 2.6106
Epoch 6/12
10000/10000 [==============================] - 44s 4ms/step - loss: 0.9238 - mae: 0.9238 - val_loss: 2.1968 - val_mae: 2.1968
Epoch 7/12
10000/10000 [==============================] - 48s 5ms/step - loss: 0.8758 - mae: 0.8758 - val_loss: 2.9211 - val_mae: 2.9211
Epoch 8/12
10000/10000 [=================

In [6]:
# Check the model interface
x = get_features(df_train.iloc[1])
y_pred = models[0].predict([x])
y_pred[0]

KeyError: 0

# Predict & submit

References: [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

Something that helped me understand this iterator was adding a pdb checkpoint inside of the for loop:

```python
import pdb; pdb.set_trace()
```

See [Python Debugging With Pdb](https://realpython.com/python-debugging-pdb/) if you want to use it and you don't know how to.


In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_features(row)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        
        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)

    # Send submissions
    env.predict(df_pred)